## Optimal portfolios of stocks, bonds, gold with different input estimation windows

In [3]:
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
import plotly.graph_objects as go
from scipy.stats import multivariate_normal as mvn
import pandas as pd

import quandl
quandl.ApiConfig.api_key = "f-5zoU2G4zzHaUtkJ7BY"

In [4]:
def tangency(means, cov, rf, short_lb):
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''

    n = len(means)
    def f(w):
        mn = w @ means
        sd = np.sqrt(w.T @ cov @ w)
        return -(mn - rf) / sd
    # Initial guess (equal-weighted)
    w0 = (1/n)*np.ones(n)
    # Constraint: fully-invested portfolio
    A = np.ones(n)
    b = 1
    cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
    bnds = [(short_lb, None) for i in range(n)] 
    # Optimization
    wgts_tangency = minimize(f, w0, bounds=bnds, constraints=cons).x
    return wgts_tangency
def gmv(cov, short_lb): 
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''    
    n = len(cov)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    if short_lb==None:
        # No position limits
        G = matrix(np.zeros((n,n)), tc="d")
        h = matrix(np.zeros(n), (n, 1), tc="d")
    else:
        # Constraint: short-sales not allowed
        G = matrix(-np.identity(n), tc="d")
        h = matrix(-short_lb * np.ones(n), (n, 1), tc="d")
    # Fully-invested constraint
    A = matrix(np.ones(n), (1, n), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b, options={'show_progress': False})
    wgts_gmv = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts_gmv    

In [5]:
# Pull the data (from sbb.py and gold.py from website codebase)
# Stocks, bonds, bills
nominal = pd.read_csv('https://www.dropbox.com/s/hgwte6swx57jqcv/nominal_sbb.csv?dl=1', index_col=['Year'])

# Gold
d = quandl.get("LBMA/GOLD")['USD (AM)']
gold = d.resample('Y').last().iloc[:-1]
gold.index = [x.year for x in gold.index]
gold.loc[1967] = d.iloc[0]
gold = gold.sort_index().pct_change().dropna()
gold.name = 'Gold'
 

data = pd.concat((nominal, gold), axis=1).dropna()
assets = ['TBills','S&P 500', 'Gold', 'Corporates', 'Treasuries']
data = data[assets]


In [17]:
 
##### Inputs
# Window length (and initial period)
window_list = [10, 15, 20]
# window_list = [5, 10, 15, 20, 25]
# window_list = [8,10,12,14,16,18,20,22,24]
# window = 10
n = len(assets)-1
raver = 5
short_lb = None
# T = len(df)-window

In [18]:
# Rolling input estimation
def backtest(df_in, window):
    df = df_in.copy()
    T = len(df)-window
    risky = assets[1:]
    print(df.columns)
    df.columns = ['rf']+['r'+str(i) for i in range(n)]
    asset_list = [str(i) for i in range(n)]
    for asset in asset_list:
        df['mn' + asset]=df['r'+asset].rolling(window).mean()
        df['sd' + asset]=df['r'+asset].rolling(window).std()

    ret_list = ['r' + asset for asset in asset_list]
    corrs = df[ret_list].rolling(window, min_periods=window).corr()

    corr_list = []
    for j, asset in enumerate(asset_list):
        for k in range(j+1,n):
            df['c'+asset+str(k)]=corrs.loc[(slice(None),'r'+asset),'r'+str(k)].values
    df['year'] = df.index
    df = df.reset_index()


    # Prepare columns for the rolling optimization output
    model_list = ['ew', 'est_all', 'est_cov', 'est_sds']
    for model in model_list:
        df['portret_'+model] = np.nan      #portret is the realized portfolio return of the 100% risky asset portfolio
        if model not in ['ew']:
            for asset in asset_list:
                df['wgt' + asset + '_' +model] = np.nan
        df['wgt_cal_'+model] =np.nan
        df['raver_portret_'+model] =np.nan #raver_portret_ is the realized return of the CAL choice of the raver investor

    mn_list = ['mn'+asset for asset in asset_list]
    sd_list = ['sd'+asset for asset in asset_list] 

    # Choose optimal portfolios each time period
    for i in np.arange(window,window+T):
        # Full estimation inputs at each point in time
        means = df[mn_list].iloc[i-1].values
        sds   = df[sd_list].iloc[i-1].values
        C  = np.identity(n)
        for j, asset in enumerate(asset_list):
            for k in range(j+1,n):
                C[j, k] = C[k, j] =    df.loc[i-1,'c'+asset+str(k)]  
        cov = np.diag(sds) @ C @ np.diag(sds)

        r = df.loc[i,'rf']
        ##### Note: all portfolio weights considered to be beginning of period weights
        ##### (so multiply by contemporaneous realized returns)
        # Full estimation tangency portfolio
        model = 'est_all'
        wgts = tangency(means,cov,r,short_lb)
        for j, asset in enumerate(asset_list):
            df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
        df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
        df.loc[i,'wgt_cal_'+model] = (wgts @ means - r) / (raver * (wgts @ cov @ wgts))
        df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
        df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
        df.loc[i,'expret_'+model] = means @ wgts
        df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

        # Estimate only covariance matrix
        model = 'est_cov'
        wgts = gmv(cov,short_lb)
        for j, asset in enumerate(asset_list):
            df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
        df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
        df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
        df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
        df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)
        df.loc[i,'expret_'+model] = means.mean()
        df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

        # Estimate only standard deviations in covariance matrix
        model = 'est_sds'
        for j, asset in enumerate(asset_list):
            for k in range(j+1,n):
                cov[j, k] = cov[k, j] = 0.0
        wgts = gmv(cov,short_lb)
        for j, asset in enumerate(asset_list):
            df.loc[i,'wgt'+asset+'_' + model] = wgts[j]
        df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
        df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))		
        df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
        df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)    
        df.loc[i,'expret_'+model] = means.mean()
        df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)

        # Equal-weighted portfolio
        model = 'ew'
        for j, asset in enumerate(asset_list):
            cov[j,j] = (sds.mean())**2
        wgts = (1/n)*np.ones(n)
        df.loc[i,'portret_'+model] = df.loc[i,ret_list].values @ wgts
        df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
        df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
        df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)   
        df.loc[i,'expret_'+model] = means.mean()
        df.loc[i,'sd'+model] = np.sqrt(wgts @ cov @ wgts)    
    
    portret_list = ['raver_portret_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
    return df[portret_list]

In [19]:

portret_list = ['raver_portret_' +  model for model in ['est_all', 'est_cov', 'est_sds','ew']]
idx = pd.MultiIndex.from_product([window_list,['est_all', 'est_sd_corr', 'est_sd','est_none']], names=["window", "strategy"])
df_rets = pd.DataFrame(dtype='float', columns=idx, index=np.arange(len(data)))

for w in window_list:
    df_rets.loc[:,(w,slice(None))] = backtest(data, w).values


Index(['TBills', 'S&P 500', 'Gold', 'Corporates', 'Treasuries'], dtype='object')
Index(['TBills', 'S&P 500', 'Gold', 'Corporates', 'Treasuries'], dtype='object')
Index(['TBills', 'S&P 500', 'Gold', 'Corporates', 'Treasuries'], dtype='object')


In [20]:
model_list = ['est_all', 'est_sd_corr', 'est_sd','est_none']
# Restrict comparison returns to the same time period
df_rets = df_rets.dropna()
stats = df_rets.describe()
sr_df = pd.DataFrame(dtype=float, columns = ['sr','avg_ret','sd_ret'], index = idx)
r = data.reset_index()[-len(df_rets):].TBills.mean()
for w in window_list:
    for model in model_list:
        sr_df.loc[(w,model),'sr'] = (stats.loc['mean',(w,model)] - r)/stats.loc['std',(w,model)]
        sr_df.loc[(w,model),'avg_ret'] = stats.loc['mean',(w,model)]
        sr_df.loc[(w,model),'sd_ret'] = stats.loc['std',(w,model)]


In [24]:
import plotly.express as px
label_dict = {'est_none': 'Est-None',
            'est_all': 'Est-All',
            'est_sd_corr': 'Est-SD-Corr',
            'est_sd': 'Est-SD'}

newdf = sr_df.reset_index().copy()
newdf['strategy'] = newdf['strategy'].apply(lambda y: label_dict[y])
# print(x)
fig = go.Figure()
fig = px.histogram(newdf, x="strategy", y="sr",
             color='window', barmode='group', histfunc='avg',
             height=400)
fig.layout.yaxis["title"] = "Sharpe ratio"
fig.layout.xaxis["title"] = "Strategy"             
fig.show()

## Plot avg SR as a function of estimation window for each strategy

In [28]:
# Plot as function of model / separate plots for horizon
import plotly.graph_objects as go
def histplot(d, w):
    df = d.loc[(w,slice(None)),:].copy()
    df=df.reset_index()
    string ="Window: %{customdata[0]} <br>"
    string += "Sharpe ratio: %{y:0.3f}<br>"
    string += "Average return: %{customdata[1]:0.1%}<br>"
    string += "SD(return): %{customdata[2]:0.1%}<br>"
    string += "<extra></extra>"
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df['strategy'], y=df['sr'], customdata=df[['window','avg_ret','sd_ret']], hovertemplate=string))
    fig.layout.yaxis["title"] = "Sharpe ratio"
    fig.layout.xaxis["title"] = "Strategy"
    fig.update_yaxes(range=[0, 1])
    fig.update_layout(title_text='Estimation window is: ' + str(w) + ' years')
    fig.show()
for w in window_list:
    histplot(sr_df, w)

## Plot average SR as function of estimation window for each strategy

In [ ]:
import plotly.graph_objects as go
def histplot(d):
    d=d.reset_index()
    string ="Strategy: %{customdata[0]} <br>"
    string += "Sharpe ratio: %{y:0.3f}<br>"
    string += "Average return: %{customdata[1]:0.1%}<br>"
    string += "SD(return): %{customdata[2]:0.1%}<br>"
    string += "<extra></extra>"
    fig = go.Figure()
    fig.add_trace(go.Bar(x=d['window'], y=d['sr'], customdata=d[['strategy','avg_ret','sd_ret']], hovertemplate=string))
    fig.layout.yaxis["title"] = "Sharpe ratio"
    fig.layout.xaxis["title"] = "Strategy"
    fig.update_yaxes(range=[0, 1])
    fig.show()
for model in model_list:
    histplot(sr_df.loc[(slice(None),model),:])